In [ ]:
import cv2
import os
import csv
import numpy as np
import pandas as pd
import mediapipe as mp
from scipy.spatial import distance as dist
from skimage.feature import local_binary_pattern


In [ ]:
# LBP parameters
radius = 1
n_points = 8 * radius
method = 'uniform'

# Landmarks indices for eyes and mouth (MediaPipe)
LEFT_EYE_IDX = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_IDX = [362, 385, 387, 263, 373, 380]
MOUTH_IDX = [61,191,13,415,291,324,14,95]  # example indices

# Input dataset folder
dataset_folders = {
    1: "/home/diya/Downloads/cv/datasets/banudeep/nthuddd2/drowsy",
    0: "/home/diya/Downloads/cv/datasets/banudeep/nthuddd2/notdrowsy"
}

# Output CSV file
output_csv = "lbp_features_fullface.csv"

mp_face_mesh = mp.solutions.face_mesh

def extract_roi(image, points):
    """Crop ROI from image using landmark points"""
    x_min = min([p[0] for p in points])
    x_max = max([p[0] for p in points])
    y_min = min([p[1] for p in points])
    y_max = max([p[1] for p in points])
    return image[y_min:y_max, x_min:x_max]

def compute_lbp_histogram(roi):
    """Compute LBP and normalized histogram with fixed 59 bins"""
    if roi.size == 0:
        return np.zeros(59)
    lbp = local_binary_pattern(roi, n_points, radius, method)
    hist, _ = np.histogram(lbp.ravel(), bins=59, range=(0, 59), density=True)
    return hist


# Open CSV for writing
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    # CSV header: filename + label + LBP bins
    n_bins = 59  # for uniform LBP with 8 neighbors
    header = ["filename", "label"] + [f"LBP_{i}" for i in range(n_bins*3)]  # left eye, right eye, mouth
    writer.writerow(header)

    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True) as face_mesh:
        for label, folder in dataset_folders.items():
            print(f"Processing label {label}, folder: {folder}")
            for filename in os.listdir(folder):
                if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
                    continue

                img_path = os.path.join(folder, filename)
                image = cv2.imread(img_path)
                if image is None:
                    print(f"Warning: {filename} could not be read. Writing NaNs...")
                    writer.writerow([filename, label] + [np.nan]*(n_bins*3))
                    continue

                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                h, w = image.shape[:2]

                results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                if results.multi_face_landmarks:
                    landmarks = results.multi_face_landmarks[0].landmark

                    # Extract ROIs
                    left_eye_points = [(int(landmarks[i].x*w), int(landmarks[i].y*h)) for i in LEFT_EYE_IDX]
                    right_eye_points = [(int(landmarks[i].x*w), int(landmarks[i].y*h)) for i in RIGHT_EYE_IDX]
                    mouth_points = [(int(landmarks[i].x*w), int(landmarks[i].y*h)) for i in MOUTH_IDX]

                    left_eye_roi = extract_roi(gray, left_eye_points)
                    right_eye_roi = extract_roi(gray, right_eye_points)
                    mouth_roi = extract_roi(gray, mouth_points)

                    # Compute LBP histograms
                    left_hist = compute_lbp_histogram(left_eye_roi)
                    right_hist = compute_lbp_histogram(right_eye_roi)
                    mouth_hist = compute_lbp_histogram(mouth_roi)

                    # Concatenate histograms
                    feature_vector = np.concatenate([left_hist, right_hist, mouth_hist])

                    # Write filename, label, and LBP features
                    writer.writerow([filename, label] + feature_vector.tolist())
                    print(f"{filename} processed. LBP features extracted.")
                else:
                    writer.writerow([filename, label] + [np.nan]*(n_bins*3))
                    print(f"No landmarks detected for {filename}. Writing NaNs.")
